# Traitements sur les vaccins

### Données

Trois jeux de données :

- juin COCONEL
- vague 1 & 2 de TRACTRUST (novembre et mai)

Plus le traitement médiatique dans les journaux avec Europresse pour le cadrage

In [1]:
import pandas as pd
import pyreadstat
import pyshs
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm
import seaborn as sns


import researchpy as rp
from scipy.stats import chi2_contingency
from scipy import stats

def labels(x,meta):
    return meta.value_labels[meta.variable_to_label[x]]
def question(x,meta):
    return meta.column_names_to_labels[x]

### Chargement des trois jeux de données

Quelques limites de comparaison :
- pas la même question COCONEL et TRACTRUST
- pas de question sur la confiance dans COCONEL

In [12]:
data1, meta1 = pyreadstat.read_sav("Questionnaire chloro/Données/Data_CORO_V14.sav")
data2, meta2 = pyreadstat.read_sav('Questionnaire CS/conseilscientifique/SujetA.sav')
data3, meta3 = pyreadstat.read_sav('Questionnaire CS/conseilscientifique/Data_118230.sav')

data1["vague"] = "COCONEL"
data2["vague"] = "TRACTRUST1"
data3["vague"] = "TRACTRUST2"

# Ajout des variables recodées pour la vague 2

# Variables générales

data1["sexe"] = data1["SEXE"].replace({1:"Homme",2:"Femme"})
data2["sexe"] = data2["SEXE"].replace({1:"Homme",2:"Femme"})
data3["sexe"] = data3["SEXE"].replace({1:"Homme",2:"Femme"})

data1["age"] = pd.cut(data1["AGE"],[17,25,35,45,55,65,100])
data2["age"] = pd.cut(data2["AGE"],[17,25,35,45,55,65,100])
data3["age"] = pd.cut(data3["AGE"],[17,25,35,45,55,65,100])

data1["age2"] = pd.cut(data1["AGE"],[17,35,55,70,100],labels=["17-34","35-54","55-79","70-100"])
data2["age2"] = pd.cut(data2["AGE"],[17,35,55,70,100],labels=["17-34","35-54","55-79","70-100"])
data3["age2"] = pd.cut(data3["AGE"],[17,35,55,70,100],labels=["17-34","35-54","55-79","70-100"])

def reco(x):
    if x in [1,2]:
        return x
    if x in [3,4,5]:
        return "3-5"
    return "6 et +"
data1["foyer"] = data1["FOYER"].apply(reco)
data2["foyer"] = data2["FOYER"].apply(reco)
data3["foyer"] = data3["FOYER"].apply(reco)

reco = {1.0: 'Catholique',
 2.0: 'Catholique',
 3.0: 'Protestant ou évangélique',
 4.0: 'Musulman',
 5.0: 'Juif',
 6.0: 'Autre religion',
 7.0: 'Sans religion',
 8.0: 'Vous ne souhaitez pas répondre'}
#data1["religion"] = data1["REL1"].replace(reco)
data2["religion"] = data2["REL1"].replace(reco)
data3["religion"] = data3["REL1"].replace(reco)

data1["csp"] = data1["PI1"].replace(labels("PI1",meta1))
data2["csp"] = data2["PI1"].replace(labels("PI1",meta2))
data3["csp"] = data3["PI1"].replace(labels("PI1",meta3))

reco = {1:"1 - inf bac",2:"1 - inf bac",3:"1 - inf bac",4:"1 - inf bac",
        5:"1 - inf bac", 6:"1 - inf bac",7:"1 - inf bac",8:"2 - bac",9:"2 - bac",
        10:"3 - sup bac",11:"3 - sup bac"}
data1["diplome"] = data1["DIPL2"].replace(reco)
data2["diplome"] = data2["DIPL2"].replace(reco)
data3["diplome"] = data3["DIPL2"].replace(reco)

reco = {1.0: '1 - 0-1000€',2.0: '2 - 1000-2000€',
        3.0: '2 - 1000-2000€',4.0: '3 - 2000-4000€',
        5.0: '3 - 2000-4000€',6.0: '4 - 4000€ and more',7.0: '5 - NA'}
data1["revenus"] = data1["REV"].replace(reco)
data2["revenus"] = data2["REV"].replace(reco)
data3["revenus"] = data3["REV"].replace(reco)

reco = {1.0: '1 - Favorable',
 2.0: '1 - Favorable',
 3.0: '2 - Défavorable',
 4.0: '2 - Défavorable'}

data1["vac_general"] = data1['OA202'].replace(reco)
data2["vac_general"] = data2['QA32'].replace(reco)
data3["vac_general"] = data3['OA29'].replace(reco)


reco = {1.0: '1 - Tout à fait confiance',
 2.0: '2 - Plutôt confiance',
 3.0: '3 - Plutôt pas confiance',
 4.0: '5 - Pas du tout confiance',
 5.0: '5 - Vous ne savez pas'}


# Variables science
reco = {1.0: '1 - Yes', 2.0: '1 - Yes', 3.0: '2 - No', 4.0: '2 - No'}
data1["interetscience"] = data1["OA194"].replace(reco)
data2["interetscience"] = data2["QA18"].replace(reco)
data3["interetscience"] = data3["OA19"].replace(reco)

reco = {1.0: '1 - Plus de bien que de mal',
 2.0: '2 - A peu près autant de bien que de mal',
 3.0: '3 - Plus de mal que de bien',
 4.0: '4 - Vous ne savez pas'}
data1["apportscience"] = data1["OA195"].replace(reco)
reco = {1.0: '1 - Plus de bien que de mal',
 2.0: '3 - Plus de mal que de bien',
 3.0: '2 - A peu près autant de bien que de mal',
 4.0: '4 - Vous ne savez pas'}
data2["apportscience"] = data2["QA19"].replace(reco)
data3["apportscience"] = data3['OA20'].replace(reco)

reco = {1.0: '1 - Confiance',
 2.0: '1 - Confiance',
 3.0: '2 - Pas confiance',
 4.0: '2 - Pas confiance',
 5.0: '3 - Vous ne savez pas'}

#data1["confiancescience"] = data1['OA221'].replace(reco) ON A PAS CET ITEM DANS COCONEL
data2["confiancescience"] = data2['QA9_r5'].replace(reco)
data3["confiancescience"] = data3['OA8_r5'].replace(reco)

data2["confiancegouvernement"] = data2['QA9_r9'].replace(reco)
data3["confiancegouvernement"] = data3['OA8_r9'].replace(reco)



reco1 = {1:1,2:1,3:0,4:0,5:0}
data2["ind_confiance"] = (data2[["QA9_r%d"%i for i in range(1,13)]].replace(reco1).sum(axis=1)/(12*1))
data2["ind_confianceC"] = pd.qcut(data2["ind_confiance"],[0,0.25,0.5,0.75,1],["Q1 - Low","Q2","Q3","Q4 - High"])
data3["ind_confiance"] = (data3[["OA8_r%d"%i for i in range(1,13)]].replace(reco1).sum(axis=1)/(12*1))
data3["ind_confianceC"] = pd.qcut(data3["ind_confiance"],[0,0.25,0.5,0.75,1],["Q1 - Low","Q2","Q3","Q4 - High"])


# Variables politique

data1["parti"] = data1["PROX"].replace(labels("PROX",meta1))
data2["parti"] = data2["PROX"].replace(labels("PROX",meta2))
data3["parti"] = data3["PROX"].replace(labels("PROX",meta3))

reco = {1.0: '5 - Extrême gauche', 2.0: '5 - Extrême gauche', 3.0: '5 - Extrême gauche', 4.0: '5 - Extrême gauche',
 5.0: '3 - Gauche', 6.0: '3 - Gauche', 7.0: '3 - Gauche',
 8.0: '1 - Centre', 9.0: '1 - Centre', 10.0: '1 - Centre', 11.0: '1 - Centre',
 12.0: '4 - Droite', 13.0: '4 - Droite',
 14.0: '6 - Extrême droite', 15.0: '6 - Extrême droite', 16.0: '6 - Extrême droite', 17.0: '6 - Extrême droite',
 18.0: '7 - Une autre formation politique',
 19.0: '2 - Aucune formation politique'}

data1["PROXPARTI"] = data1["PROX"].replace(reco)
data2["PROXPARTI"] = data2["PROX"].replace(reco)
data3["PROXPARTI"] = data3["PROX"].replace(reco)


reco = {1:"Frequently",2:"Frequently",3:"Rarely",4:"Rarely",5:"Rarely"}
data1["infopol"] = data1["OA191"].replace(reco)
data2["infopol"] = data2["QA2"].replace(reco)
data3["infopol"] = data3["OA2"].replace(reco)

reco = {1:1,2:1,3:0,4:0,5:0}
data1["politisation"] = data1["OA190"].replace(reco) + data1["OA191"].replace(reco) + data1["OA193"].replace(reco) 
data1["politisationC"]  = data1["politisation"].replace({3:"3 - High",2:"2 - Average",1:"1 - Low",0:"1 - Low"})
data2["politisation"] = data2["QA1"].replace(reco) + data2["QA2"].replace(reco) + data2["QA4"].replace(reco) 
data2["politisationC"]  = data1["politisation"].replace({3:"3 - High",2:"2 - Average",1:"1 - Low",0:"1 - Low"})
data3["politisation"] = data3["OA1"].replace(reco) + data3["OA2"].replace(reco) + data3["OA4"].replace(reco) 
data3["politisationC"]  = data3["politisation"].replace({3:"3 - High",2:"2 - Average",1:"1 - Low",0:"1 - Low"})


# Variable chloroquine -> attention, dans la première vague on a fait un filtre sur le fait de savoir si oui ou non
# ils avaient entendu parler du débat

reco = {1:"1 - HC est un bon traitement",2:"2 - HC n'est pas un bon traitement",
        3:"3 - Conaissances insuffisantes",4:"4 - NSP"}
data1["HC_savoir"] = data1["OA200"].replace(labels("OA200",meta1))
# Croyance dans la chloroquine vague 1
reco = {3:"Incertains",4:"Sans avis",1:"Croyant",2:"Non croyant"}
data1["profil_hc"] = data1["OA201"].replace(reco)
data1["profil_hc"] = data1["profil_hc"].fillna("Ne connaissent pas")
data1["Croyant"] = data1["profil_hc"].apply(lambda x : 1 if x=="Croyant" else 0)

data1["HC"] = data1["OA201"].replace(reco)
data2["HC"] = data2["QA29"].replace(reco)
data3["HC"] = data3["OA27"].replace(reco)

reco = {1.0: 'Oui',2.0: 'Oui', 3.0: 'Non',4.0: 'Non'}
data1["vaccincovid"] = data1["OA27"].replace(reco)
data2["vaccincovid"] = data2["QA30"].replace(reco)
reco = {1.0: 'Oui',2.0: 'Oui', 3.0: 'Non',4.0: 'Non',5.0:"Oui"}
data3["vaccincovid"] = data3["OA28"].replace(reco)

reco = {1.0: 'Oui',2.0: 'Oui', 3.0: 'Non',4.0: 'Non'}
data1["vaccin"] = data1["OA202"].replace(reco)
data2["vaccin"] = data2["QA32"].replace(reco)
data3["vaccin"] = data3["OA29"].replace(reco)

# version complète

reco = {1.0: '1 - Oui, certainement',
 2.0: '1 - Oui, probablement',
 3.0: '2 - Non, probablement pas',
 4.0: '3 - Non, certainement pas'}
data1["vaccincovidF"] = data1["OA27"].replace(reco)
data2["vaccincovidF"] = data2["QA30"].replace(reco)
reco = {1.0: '1 - Oui certainement',
 2.0: '2 - Oui, probablement',
 3.0: '3 - Non, probablement pas',
 4.0: '4 - Non, certainement pas',
 5.0: '5 - Vous êtes déjà vacciné (une dose ou plus)'}
data3["vaccincovidF"] = data3["OA28"].replace(reco)


reco = {1.0: '1 - Très favorable',
 2.0: '2 - Plutôt favorable',
 3.0: '3 - Plutôt pas favorable',
 4.0: '4 - Pas du tout favorable'}
data1["vaccinF"] = data1["OA202"].replace(reco)
data2["vaccinF"] = data2["QA32"].replace(reco)
data3["vaccinF"] = data3["OA29"].replace(reco)

In [9]:
labels("OA29",meta3)

{1.0: 'Très favorable',
 2.0: 'Plutôt favorable',
 3.0: 'Plutôt pas favorable',
 4.0: 'Pas du tout favorable'}

## non intention de se faire vacciner (plutôt pas ou certainement pas à la question sur les intentions de vaccination contre la covid vs le reste). Les variables indépendantes seraient : sexe, age, niveau de revenu, niveau d’éducation

In [28]:
# Variable binaire non
data1["vaccincovidN"] = data1["vaccincovid"].apply(lambda x : 1 if x == "Non" else 0)
data2["vaccincovidN"] = data2["vaccincovid"].apply(lambda x : 1 if x == "Non" else 0)
data3["vaccincovidN"] = data3["vaccincovid"].apply(lambda x : 1 if x == "Non" else 0)

tab = {
        "COCONEL":pyshs.regression_logistique(data1,"vaccincovidN",["sexe","age2","revenus","diplome"],"RED"),
        "TRACTRUST1":pyshs.regression_logistique(data2,"vaccincovidN",["sexe","age2","revenus","diplome"],"RED"),
        "TRACTRUST2":pyshs.regression_logistique(data2,"vaccincovidN",["sexe","age2","revenus","diplome"],"RED")}
pd.concat(tab,axis=1)

COCONEL                             \
                              Odds Ratio        p            IC 95%   
Variable   Modalité                                                   
.Intercept                          2.23   0.0***  2.23 [1.58-3.15]   
age2       17-34                     ref                              
           35-54                    0.69  0.002**  0.69 [0.55-0.87]   
           55-79                    0.38   0.0***  0.38 [0.29-0.51]   
           70-100                   0.21   0.0***  0.21 [0.15-0.29]   
diplome    1 - inf bac               ref                              
           2 - bac                  0.98    0.863  0.98 [0.75-1.27]   
           3 - sup bac              0.74   0.012*  0.74 [0.58-0.94]   
revenus    1 - 0-1000€               ref                              
           2 - 1000-2000€            0.7    0.04*  0.70 [0.50-0.98]   
           3 - 2000-4000€           0.55   0.0***  0.55 [0.39-0.76]   
           4 - 4000€ and more       0.38   0.0***  0.38 [0.24-0.60]   
           5 - NA                   0.59   0.013*  0.59 [0.39-0.90]   
sexe       Femme                     ref                              
           Homme                    0.66   0.0***  0.66 [0.55-0.80]   

                              TRACTRUST1                             \
                              Odds Ratio        p            IC 95%   
Variable   Modalité                                                   
.Intercept                          3.88   0.0***  3.88 [2.26-6.64]   
age2       17-34                     ref                              
           35-54                    0.62  0.009**  0.62 [0.44-0.89]   
           55-79                    0.27   0.0***  0.27 [0.18-0.40]   
           70-100                   0.16   0.0***  0.16 [0.10-0.26]   
diplome    1 - inf bac               ref                              
           2 - bac                  0.87    0.464  0.87 [0.60-1.27]   
           3 - sup bac              0.58  0.002**  0.58 [0.41-0.82]   
revenus    1 - 0-1000€               ref                              
           2 - 1000-2000€           1.24     0.41  1.24 [0.74-2.09]   
           3 - 2000-4000€           1.22    0.456  1.22 [0.73-2.04]   
           4 - 4000€ and more       1.01    0.982  1.01 [0.51-1.97]   
           5 - NA                   0.92    0.799  0.92 [0.51-1.69]   
sexe       Femme                     ref                              
           Homme                    0.44   0.0***  0.44 [0.34-0.58]   

                              TRACTRUST2                             
                              Odds Ratio        p            IC 95%  
Variable   Modalité                                                  
.Intercept                          3.88   0.0***  3.88 [2.26-6.64]  
age2       17-34                     ref                             
           35-54                    0.62  0.009**  0.62 [0.44-0.89]  
           55-79                    0.27   0.0***  0.27 [0.18-0.40]  
           70-100                   0.16   0.0***  0.16 [0.10-0.26]  
diplome    1 - inf bac               ref                             
           2 - bac                  0.87    0.464  0.87 [0.60-1.27]  
           3 - sup bac              0.58  0.002**  0.58 [0.41-0.82]  
revenus    1 - 0-1000€               ref                             
           2 - 1000-2000€           1.24     0.41  1.24 [0.74-2.09]  
           3 - 2000-4000€           1.22    0.456  1.22 [0.73-2.04]  
           4 - 4000€ and more       1.01    0.982  1.01 [0.51-1.97]  
           5 - NA                   0.92    0.799  0.92 [0.51-1.69]  
sexe       Femme                     ref                             
           Homme                    0.44   0.0***  0.44 [0.34-0.58]

## 2. Spire et Bajos mettent pas ensemble ceux qui n'ont plutot pas l'intention de se faire vacciner et ceux qui disent "certainement pas". Pour voir si leurs résultats sont spécifiques à la période ou s'ils traduisent une constante pour la période, j'aimerais faire la même régression que ci-dessus sur les 3 BDD sauf que ce serait les "certainement pas" vs le reste.

In [24]:
reco = {1.0: 0,2.0: 0, 3.0: 0,4.0: 1}
data1["vaccincovidCP"] = data1["OA27"].replace(reco)
data2["vaccincovidCP"] = data2["QA30"].replace(reco)
reco = {1.0: 0,2.0: 0, 3.0: 0,4.0: 1,5.0:0}
data3["vaccincovidCP"] = data3["OA28"].replace(reco)

In [29]:
tab = {
        "COCONEL":pyshs.tri_a_plat(data1,"vaccincovidCP","RED"),
        "TRACTRUST1":pyshs.tri_a_plat(data2,"vaccincovidCP","RED"),
        "TRACTRUST2":pyshs.tri_a_plat(data3,"vaccincovidCP","RED")}
pd.concat(tab,axis=1)

COCONEL                        TRACTRUST1                  \
      Effectif redressé Pourcentage (%) Effectif redressé Pourcentage (%)   
0.0              1734.9            86.5             768.7            75.8   
1.0               271.1            13.5             245.3            24.2   
Total            2006.0           100.0            1014.0           100.0   

             TRACTRUST2                  
      Effectif redressé Pourcentage (%)  
0.0               885.4            88.1  
1.0               119.6            11.9  
Total            1005.0           100.0

Il y a un truc étrange sur la vague 1 de TRACTRUST (quasiment le double)

In [27]:
tab = {
        "COCONEL":pyshs.regression_logistique(data1,"vaccincovidCP",["sexe","age2","revenus","diplome"],"RED"),
        "TRACTRUST1":pyshs.regression_logistique(data2,"vaccincovidCP",["sexe","age2","revenus","diplome"],"RED"),
        "TRACTRUST2":pyshs.regression_logistique(data2,"vaccincovidCP",["sexe","age2","revenus","diplome"],"RED")}
pd.concat(tab,axis=1)

COCONEL                             \
                              Odds Ratio        p            IC 95%   
Variable   Modalité                                                   
.Intercept                          0.62   0.021*  0.62 [0.41-0.93]   
age2       17-34                     ref                              
           35-54                    1.14    0.402  1.14 [0.84-1.55]   
           55-79                    0.54  0.003**  0.54 [0.36-0.81]   
           70-100                   0.18   0.0***  0.18 [0.09-0.33]   
diplome    1 - inf bac               ref                              
           2 - bac                   0.7   0.049*  0.70 [0.49-1.00]   
           3 - sup bac              0.52   0.0***  0.52 [0.37-0.73]   
revenus    1 - 0-1000€               ref                              
           2 - 1000-2000€           0.51  0.001**  0.51 [0.34-0.77]   
           3 - 2000-4000€           0.44   0.0***  0.44 [0.30-0.66]   
           4 - 4000€ and more       0.22   0.0***  0.22 [0.11-0.44]   
           5 - NA                   0.54   0.019*  0.54 [0.33-0.90]   
sexe       Femme                     ref                              
           Homme                    0.63  0.001**  0.63 [0.48-0.83]   

                              TRACTRUST1                             \
                              Odds Ratio        p            IC 95%   
Variable   Modalité                                                   
.Intercept                          1.34    0.285  1.34 [0.78-2.31]   
age2       17-34                     ref                              
           35-54                    0.72    0.072  0.72 [0.50-1.03]   
           55-79                    0.33   0.0***  0.33 [0.21-0.53]   
           70-100                   0.17   0.0***  0.17 [0.09-0.31]   
diplome    1 - inf bac               ref                              
           2 - bac                  0.83    0.375  0.83 [0.55-1.25]   
           3 - sup bac              0.49  0.001**  0.49 [0.33-0.74]   
revenus    1 - 0-1000€               ref                              
           2 - 1000-2000€           0.82    0.456  0.82 [0.48-1.38]   
           3 - 2000-4000€           0.68     0.16  0.68 [0.40-1.16]   
           4 - 4000€ and more       0.49    0.085  0.49 [0.21-1.10]   
           5 - NA                   0.65    0.192  0.65 [0.35-1.24]   
sexe       Femme                     ref                              
           Homme                    0.44   0.0***  0.44 [0.32-0.61]   

                              TRACTRUST2                             
                              Odds Ratio        p            IC 95%  
Variable   Modalité                                                  
.Intercept                          1.34    0.285  1.34 [0.78-2.31]  
age2       17-34                     ref                             
           35-54                    0.72    0.072  0.72 [0.50-1.03]  
           55-79                    0.33   0.0***  0.33 [0.21-0.53]  
           70-100                   0.17   0.0***  0.17 [0.09-0.31]  
diplome    1 - inf bac               ref                             
           2 - bac                  0.83    0.375  0.83 [0.55-1.25]  
           3 - sup bac              0.49  0.001**  0.49 [0.33-0.74]  
revenus    1 - 0-1000€               ref                             
           2 - 1000-2000€           0.82    0.456  0.82 [0.48-1.38]  
           3 - 2000-4000€           0.68     0.16  0.68 [0.40-1.16]  
           4 - 4000€ and more       0.49    0.085  0.49 [0.21-1.10]  
           5 - NA                   0.65    0.192  0.65 [0.35-1.24]  
sexe       Femme                     ref                             
           Homme                    0.44   0.0***  0.44 [0.32-0.61]

## 3.Spire et Bajos font la même chose pour les attitudes à l'égard de la vaccination en général. L'item est "êtes vous favorable aux vaccins en général' et ils font les "pas du tout" vs le reste. Je veux bien que l'on fasse pareil sur les 3 BDD (toujours avec le  sexe, age, niveau de revenu, niveau d’éducation comme variables indépendantes)

In [38]:
reco = {1.0: 0,2.0: 0, 3.0: 0,4.0: 1}
data1["vaccinDF"] = data1["OA202"].replace(reco)
data2["vaccinDF"] = data2["QA32"].replace(reco)
data3["vaccinDF"] = data3["OA29"].replace(reco)

In [39]:
tab = {
        "COCONEL":pyshs.tri_a_plat(data1,"vaccinDF","RED"),
        "TRACTRUST1":pyshs.tri_a_plat(data2,"vaccinDF","RED"),
        "TRACTRUST2":pyshs.tri_a_plat(data3,"vaccinDF","RED")}
pd.concat(tab,axis=1)

COCONEL                        TRACTRUST1                  \
      Effectif redressé Pourcentage (%) Effectif redressé Pourcentage (%)   
0.0              1835.9            91.5             932.8            92.0   
1.0               170.1             8.5              81.2             8.0   
Total            2006.0           100.0            1014.0           100.0   

             TRACTRUST2                  
      Effectif redressé Pourcentage (%)  
0.0               927.7            92.3  
1.0                77.3             7.7  
Total            1005.0           100.0

In [40]:
tab = {
        "COCONEL":pyshs.regression_logistique(data1,"vaccinDF",["sexe","age2","revenus","diplome"],"RED"),
        "TRACTRUST1":pyshs.regression_logistique(data2,"vaccinDF",["sexe","age2","revenus","diplome"],"RED"),
        "TRACTRUST2":pyshs.regression_logistique(data2,"vaccinDF",["sexe","age2","revenus","diplome"],"RED")}
pd.concat(tab,axis=1)

COCONEL                             \
                              Odds Ratio        p            IC 95%   
Variable   Modalité                                                   
.Intercept                          0.33   0.0***  0.33 [0.20-0.53]   
age2       17-34                     ref                              
           35-54                     1.0    0.983  1.00 [0.69-1.45]   
           55-79                    0.51  0.007**  0.51 [0.31-0.84]   
           70-100                   0.23   0.0***  0.23 [0.11-0.45]   
diplome    1 - inf bac               ref                              
           2 - bac                  0.61    0.03*  0.61 [0.39-0.95]   
           3 - sup bac              0.45   0.0***  0.45 [0.29-0.69]   
revenus    1 - 0-1000€               ref                              
           2 - 1000-2000€           0.54   0.014*  0.54 [0.33-0.88]   
           3 - 2000-4000€           0.55   0.013*  0.55 [0.34-0.88]   
           4 - 4000€ and more        0.2  0.001**  0.20 [0.07-0.52]   
           5 - NA                   0.66    0.169  0.66 [0.36-1.20]   
sexe       Femme                     ref                              
           Homme                    0.77    0.118  0.77 [0.56-1.07]   

                              TRACTRUST1                           TRACTRUST2  \
                              Odds Ratio       p            IC 95% Odds Ratio   
Variable   Modalité                                                             
.Intercept                          0.19  0.0***  0.19 [0.08-0.43]       0.19   
age2       17-34                     ref                                  ref   
           35-54                    1.21   0.504  1.21 [0.69-2.13]       1.21   
           55-79                    0.52   0.094  0.52 [0.24-1.12]       0.52   
           70-100                   0.33  0.026*  0.33 [0.12-0.88]       0.33   
diplome    1 - inf bac               ref                                  ref   
           2 - bac                  0.76   0.375  0.76 [0.41-1.39]       0.76   
           3 - sup bac              0.42  0.013*  0.42 [0.22-0.83]       0.42   
revenus    1 - 0-1000€               ref                                  ref   
           2 - 1000-2000€           0.98   0.951  0.98 [0.46-2.09]       0.98   
           3 - 2000-4000€            0.6   0.219  0.60 [0.27-1.35]        0.6   
           4 - 4000€ and more       0.19   0.091  0.19 [0.03-1.30]       0.19   
           5 - NA                   1.04   0.934  1.04 [0.42-2.56]       1.04   
sexe       Femme                     ref                                  ref   
           Homme                    0.78   0.325  0.78 [0.48-1.27]       0.78   

                                                         
                                    p            IC 95%  
Variable   Modalité                                      
.Intercept                     0.0***  0.19 [0.08-0.43]  
age2       17-34                                         
           35-54                0.504  1.21 [0.69-2.13]  
           55-79                0.094  0.52 [0.24-1.12]  
           70-100              0.026*  0.33 [0.12-0.88]  
diplome    1 - inf bac                                   
           2 - bac              0.375  0.76 [0.41-1.39]  
           3 - sup bac         0.013*  0.42 [0.22-0.83]  
revenus    1 - 0-1000€                                   
           2 - 1000-2000€       0.951  0.98 [0.46-2.09]  
           3 - 2000-4000€       0.219  0.60 [0.27-1.35]  
           4 - 4000€ and more   0.091  0.19 [0.03-1.30]  
           5 - NA               0.934  1.04 [0.42-2.56]  
sexe       Femme                                         
           Homme                0.325  0.78 [0.48-1.27]

## Ils ne concernent que les vagues de tractrust. Je voudrais pour chacune des 2 un tableau tout simple croisant les réponses à la question "êtes vous favorables aux vaccins en général' et celles à la question "êtes-vous défavorables à certains vaccins en particulier".

In [46]:
reco = {1.0: '1 - Très favorable',
 2.0: '2 - Plutôt favorable',
 3.0: '3 - Plutôt pas favorable',
 4.0: '4 - Pas du tout favorable'}
data2["vaccin"] = data2["QA32"].replace(reco)
data3["vaccin"] = data3["OA29"].replace(reco)
reco = {1.0: '1 - Oui', 2.0: 'Non', 3.0: 'Vous ne savez pas'}
data2["vaccinpar"] = data2["QA33"].replace(reco)
data3["vaccinpar"] = data3["OA30"].replace(reco)

In [47]:
pyshs.tableau_croise(data2,"vaccin","vaccinpar","RED")

vaccinpar,1 - Oui,Non,Vous ne savez pas,All
vaccin,,,,
1 - Très favorable,32.5 (13.5%),188.6 (78.1%),20.3 (8.4%),241.5 (100%)
2 - Plutôt favorable,139.5 (28.7%),253.5 (52.1%),93.1 (19.2%),486.1 (100%)
3 - Plutôt pas favorable,139.0 (67.7%),43.4 (21.2%),22.8 (11.1%),205.2 (100%)
4 - Pas du tout favorable,40.0 (49.3%),24.6 (30.3%),16.6 (20.4%),81.2 (100%)
All,351.1 (34.6%),510.1 (50.3%),152.8 (15.1%),1014.0 (100%)


In [48]:
pyshs.tableau_croise(data3,"vaccin","vaccinpar","RED")

vaccinpar,1 - Oui,Non,Vous ne savez pas,All
vaccin,,,,
1 - Très favorable,63.0 (17.5%),285.2 (79.2%),12.0 (3.3%),360.2 (100%)
2 - Plutôt favorable,128.2 (33.1%),207.0 (53.4%),52.4 (13.5%),387.5 (100%)
3 - Plutôt pas favorable,104.0 (57.8%),47.9 (26.6%),28.0 (15.6%),179.9 (100%)
4 - Pas du tout favorable,48.2 (62.4%),18.2 (23.5%),10.9 (14.1%),77.3 (100%)
All,343.4 (34.2%),558.3 (55.6%),103.3 (10.3%),1005.0 (100%)


Tableaux croisés du 23072021

COCONEL

In [16]:
pyshs.tableau_croise(data1,"vaccinF","vaccincovidF")

vaccincovidF,"1 - Oui, certainement","1 - Oui, probablement","2 - Non, probablement pas","3 - Non, certainement pas",All
vaccinF,,,,,
1 - Très favorable,349 (60.5%),182 (31.5%),37 (6.4%),9 (1.6%),577 (100%)
2 - Plutôt favorable,120 (12.8%),547 (58.2%),223 (23.7%),50 (5.3%),940 (100%)
3 - Plutôt pas favorable,6 (1.8%),66 (19.6%),183 (54.3%),82 (24.3%),337 (100%)
4 - Pas du tout favorable,3 (2.0%),7 (4.6%),32 (21.1%),110 (72.4%),152 (100%)
All,478 (23.8%),802 (40.0%),475 (23.7%),251 (12.5%),2006 (100%)


TRACTRUST 1

In [17]:
pyshs.tableau_croise(data2,"vaccinF","vaccincovidF")

vaccincovidF,"1 - Oui, certainement","1 - Oui, probablement","2 - Non, probablement pas","3 - Non, certainement pas",All
vaccinF,,,,,
1 - Très favorable,124 (49.0%),85 (33.6%),27 (10.7%),17 (6.7%),253 (100%)
2 - Plutôt favorable,33 (6.7%),214 (43.2%),171 (34.5%),77 (15.6%),495 (100%)
3 - Plutôt pas favorable,3 (1.6%),25 (13.2%),86 (45.3%),76 (40.0%),190 (100%)
4 - Pas du tout favorable,1 (1.3%),2 (2.6%),13 (17.1%),60 (78.9%),76 (100%)
All,161 (15.9%),326 (32.1%),297 (29.3%),230 (22.7%),1014 (100%)


TRACTRUST 2

In [18]:
pyshs.tableau_croise(data3,"vaccinF","vaccincovidF")

vaccincovidF,1 - Oui certainement,"2 - Oui, probablement","3 - Non, probablement pas","4 - Non, certainement pas",5 - Vous êtes déjà vacciné (une dose ou plus),All
vaccinF,,,,,,
1 - Très favorable,96 (26.9%),12 (3.4%),11 (3.1%),10 (2.8%),228 (63.9%),357 (100%)
2 - Plutôt favorable,55 (13.8%),107 (26.9%),38 (9.5%),33 (8.3%),165 (41.5%),398 (100%)
3 - Plutôt pas favorable,17 (9.4%),40 (22.2%),48 (26.7%),34 (18.9%),41 (22.8%),180 (100%)
4 - Pas du tout favorable,7 (10.0%),4 (5.7%),13 (18.6%),33 (47.1%),13 (18.6%),70 (100%)
All,175 (17.4%),163 (16.2%),110 (10.9%),110 (10.9%),447 (44.5%),1005 (100%)
